In [1]:
import os, subprocess
out = subprocess.run(["uname", "-a"], capture_output=True)
if "Darwin" in str(out.stdout):
    print('overriding PATH')
    os.environ['PATH'] = "/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:" +  os.environ['PATH']

overriding PATH


In [2]:
# Orchestration/Data Extraction function defs
import json

import pandas as pd

from test_analysis import *

In [88]:
def check_data_assumptions(df: pd.DataFrame):
    assert len(df[[
        df['confirmation_judge_payload.response.outcome_agrees_with_expected_outcome'] == True 
        & df['confirmation_judge_payload.response.outcome_disagrees_with_expected_outcome'] == True
    ]]) == 0
    assert len(df[[
        df['meta.judgement.permutation'] != df['meta.conversation.permutation']
    ]]) == 0
    assert len(df[[
        df['meta.judgement.test_case'] != df['meta.conversation.test_case']
    ]]) == 0
    assert len(df[[
        df['meta.judgement.test_cohort'] != df['meta.conversation.test_cohort']
    ]]) == 0
    assert len(df[[
        df['meta.judgement.hypothesis_name'] != df['meta.conversation.hypothesis_name']
    ]]) == 0

In [89]:
useful_columns = [
    'conversation', 
    'expected_outcome', 

    'meta.judgement.permutation',
    'meta.judgement.test_case',
    'meta.judgement.commit',
    'meta.judgement.hypothesis_name', 
    'meta.judgement.test_cohort',
    'meta.judgement.execution_datetime',

    'meta.conversation.permutation',
    'meta.conversation.test_case',
    'meta.conversation.commit',
    'meta.conversation.hypothesis_name', 
    'meta.conversation.test_cohort',
    'meta.conversation.execution_datetime',

    'eligibility_agent_payload.response.would_application_be_eligible',
    'eligibility_agent_payload.response.would_application_be_ineligible',
    'eligibility_agent_payload.response.would_application_be_eligible_in_part',
    'eligibility_agent_payload.response.reasoning_for_eligibility_judgement',

    'confirmation_judge_payload.response.outcome_agrees_with_expected_outcome',
    'confirmation_judge_payload.response.outcome_disagrees_with_expected_outcome',
    'confirmation_judge_payload.response.outcome_partly_agrees_with_expected_outcome',
    'confirmation_judge_payload.response.erroneous_info_given_by_eligibility_agent_without_realising',
    'confirmation_judge_payload.response.erroneous_info_given_by_eligibility_agent_but_later_realised',
    'confirmation_judge_payload.response.reasoning_for_conversation_judgement',
]
def load_results_from_json(output_dir: Path, impatient=False):
    files_to_load = output_dir.glob("**/Permutation*__judgement_*.json")
    list_of_json_dataframes = []
    for file_to_load in files_to_load:
        with file_to_load.open() as f:
            try:
                #print(f'Trying {file_to_load.absolute()}')
                list_of_json_dataframes.append(
                    pd.json_normalize(
                        json.load(f),
                        meta=useful_columns,
                        errors='raise'
                    ).set_index(
                        [
                            # Unique per conversation
                            "meta.conversation.permutation",
                            "meta.conversation.execution_datetime",
                            # Unique per judgement
                            "meta.judgement.execution_datetime",
                        ]
                    )
                )
            except json.JSONDecodeError as e:
                if impatient:
                    print(f'Skipping {file_to_load.absolute()} as it is not valid JSON')
                    continue
                else:
                    raise e

        df = pd.concat(
            list_of_json_dataframes,
            verify_integrity=True
        )
        check_data_assumptions(df) 
        return df


In [74]:
output_dir = Path('./testOutputs/child_benefit')
structured_df = load_results_from_json(output_dir, impatient=True)

In [ ]:
structured_df.columns

Index(['conversation', 'expected_outcome', 'meta.conversation.test_case',
       'meta.conversation.commit', 'meta.conversation.hypothesis_name',
       'meta.conversation.test_cohort', 'meta.judgement.permutation',
       'meta.judgement.test_case', 'meta.judgement.commit',
       'meta.judgement.hypothesis_name', 'meta.judgement.test_cohort',
       'eligibility_agent_payload.will_continue',
       'eligibility_agent_payload.scheduling',
       'eligibility_agent_payload.parts', 'eligibility_agent_payload.id',
       'eligibility_agent_payload.name',
       'eligibility_agent_payload.response.would_application_be_eligible',
       'eligibility_agent_payload.response.would_application_be_ineligible',
       'eligibility_agent_payload.response.would_application_be_eligible_in_part',
       'eligibility_agent_payload.response.reasoning_for_eligibility_judgement',
       'confirmation_judge_payload.will_continue',
       'confirmation_judge_payload.scheduling',
       'confirmation_judge

In [ ]:
structured_df[
    structured_df['confirmation_judge_payload.response.outcome_partly_agrees_with_expected_outcome']== True
][[
    "meta.judgement.permutation",
    'confirmation_judge_payload.response.reasoning_for_conversation_judgement',
]]

,,,meta.judgement.permutation,confirmation_judge_payload.response.reasoning_for_conversation_judgement
meta.conversation.permutation,meta.conversation.execution_datetime,meta.judgement.execution_datetime,,
8,2026-02-18T14:07:48.347279,2026-02-18T16:01:52.616627,8,"The agent correctly assessed the basic eligibility criteria for Child Benefit (responsibility, child's age, residency, and living arrangements) and correctly concluded that the applicant is eligible to apply. However, the agent failed critically by not investigating income levels or mentioning the High Income Child Benefit Charge (HICBC).\n\nThe expected outcome is ""Eligible (but subject to tax charge)"" which indicates this is the ""High Earner"" test case where one partner earns £75,000 per year - well over the £60,000 threshold for HICBC. The agent should have:\n1. Asked about the income of both partners\n2. Identified that income over £60,000 triggers a tax charge\n3. Explained that at £75,000, they would face a significant HICBC (approximately 75% of the benefit would be clawed back via tax)\n4. Mentioned the option to claim but opt out of receiving payments\n\nBy giving an unqualified ""you would be eligible"" response without addressing the tax charge implications, the agent provided misleading information by omission. The applicant would face an unexpected tax bill if they followed this advice without understanding the HICBC consequences. This is a material omission that makes the guidance incomplete and potentially harmful to the user's financial situation."
24,2026-02-18T14:07:48.347279,2026-02-18T16:01:52.616627,24,"The agent's response partially agrees with the expected outcome but contains a critical omission that constitutes erroneous information by omission. \n\nThe agent correctly determined that the parent is currently eligible for Child Benefit (which aligns with ""Eligible until week 12"" since Tom has only been in residential accommodation for 10 weeks). However, the agent failed to mention the crucial 12-week time limit that applies to children in residential accommodation paid for by the local council.\n\nAccording to the expected outcome and the test case criteria, children in residential accommodation retain eligibility for only 12 weeks, after which eligibility ceases. Tom has been there for 10 weeks, meaning the parent has only 2 weeks of eligibility remaining.\n\nThe agent's statement ""You should be able to apply for or continue receiving Child Benefit for Tom"" is misleading because it suggests ongoing, indefinite eligibility without mentioning this imminent time limit. This is a material omission that would leave the parent uninformed about a critical aspect of their eligibility status.\n\nThe agent gave erroneous information by presenting eligibility as open-ended when it is actually time-limited and about to expire. This failure to communicate the 12-week rule means the agent did not provide complete or accurate guidance."
27,2026-02-18T14:07:48.347279,2026-02-18T16:01:52.616627,27,"The agent reached a partially correct conclusion but failed in critical ways. While the basic eligibility determination is technically correct (a minor CAN claim Child Benefit for their own child), the agent failed to:\n\n1. **Ask about the applicant's age** - This is essential information that was never gathered, despite being crucial to the scenario.\n\n2. **Provide important guidance about financial implications** - The expected outcome specifically notes this is ""Eligible (Though potentially financially suboptimal)."" The test case clearly states that ""You'll usually get more if [someone responsible for you] claims... on your behalf"" and that if the minor claims, their parents might lose the Child Benefit they get for the minor themselves.\n\n3. **Give complete advice** - The agent should have informed the applicant about alternative claiming arrangements that would be more financially beneficial.\n\nThe agent's response is misleading by omission. W

In [97]:
structured_df[
    structured_df['confirmation_judge_payload.response.outcome_agrees_with_expected_outcome']== False
][[
    "meta.judgement.permutation",
    'confirmation_judge_payload.response.reasoning_for_conversation_judgement',
]]

,,,meta.judgement.permutation,confirmation_judge_payload.response.reasoning_for_conversation_judgement
meta.conversation.permutation,meta.conversation.execution_datetime,meta.judgement.execution_datetime,,
23,2026-02-18T14:07:48.347279,2026-02-18T16:01:52.616627,23,"The agent concluded Kevin is INELIGIBLE for Child Benefit, but the expected outcome is ELIGIBLE. The agent made a critical error in applying the eligibility rules.\n\nAccording to https://www.gov.uk/child-benefit-child-lives-with-someone-else, when a child doesn't live with you, you CAN claim Child Benefit if: (1) you contribute at least the Child Benefit rate towards the child's upkeep, AND (2) no one else is claiming Child Benefit for that child.\n\nThe agent correctly calculated that Kevin's contribution of £120 every 4 weeks (£30/week) exceeds the required threshold of £25.60/week. However, the agent incorrectly concluded Kevin is ineligible by applying a non-existent ""priority"" rule.\n\nThe agent stated: ""Even if her mother isn't currently claiming Child Benefit, she would have the stronger entitlement as the primary carer with whom your daughter lives."" This is incorrect. The rules don't establish a hierarchy based on who has day-to-day care when no one is currently claiming. If the mother is NOT claiming, Kevin IS eligible because he meets both criteria: sufficient contribution and no one else claiming.\n\nKevin indicated he doesn't know if the mother is claiming. The agent should have clarified this crucial point and explained that Kevin's eligibility depends on whether the mother is actually claiming, rather than categorically stating he is ineligible regardless. The agent gave erroneous information without realizing the error."
34,2026-02-18T14:07:48.347279,2026-02-18T16:01:52.616627,34,"The expected outcome is ""Not Eligible"" but the agent concluded the application is ""eligible in part"" (partial eligibility). While the agent correctly identified all the relevant facts - that the son has been in custody for 10 weeks (exceeding the 8-week limit), that Child Benefit stops after 8 weeks in custody, and that the mother is ""not currently eligible"" - the agent's final determination was incorrect.\n\nThe agent framed this as ""partial eligibility"" because the mother could claim again when her son is released. However, this is not a case of partial eligibility. The question at hand is about current eligibility given the present circumstances. The mother is simply not eligible right now because her son has been in custody for more than 8 weeks. The fact that she may become eligible in the future when circumstances change doesn't make her ""partially eligible"" now - it just means her future eligibility could be different.\n\nThe agent's payload also marked ""would_application_be_eligible_in_part: true"" which contradicts the expected outcome of ""Not Eligible."" The correct determination should have been that the applicant is not eligible for Child Benefit at this time, with helpful guidance that eligibility can restart upon the child's release. The agent conflated current ineligibility with future potential eligibility, leading to an incorrect outcome determination."
8,2026-02-18T14:07:48.347279,2026-02-18T16:01:52.616627,8,"The agent correctly assessed the basic eligibility criteria for Child Benefit (responsibility, child's age, residency, and living arrangements) and correctly concluded that the applicant is eligible to apply. However, the agent failed critically by not investigating income levels or mentioning the High Income Child Benefit Charge (HICBC).\n\nThe expected outcome is ""Eligible (but subject to tax charge)"" which indicates this is the ""High Earner"" test case where one partner earns £75,000 per year - well over the £60,000 threshold for HICBC. The agent should have:\n1. Asked about the income of both partners\n2. Identified that income over £60,000 triggers a tax charge\n3. Explained that at £75,000, they would face a significant HIC

In [96]:
(
    structured_df
    # [
        # structured_df['confirmation_judge_payload.response.outcome_partly_agrees_with_expected_outcome']== True
        # structured_df['confirmation_judge_payload.response.outcome_agrees_with_expected_outcome']== False
    # ]
    .value_counts([
        'confirmation_judge_payload.response.outcome_agrees_with_expected_outcome',
        'confirmation_judge_payload.response.erroneous_info_given_by_eligibility_agent_without_realising',
        'confirmation_judge_payload.response.erroneous_info_given_by_eligibility_agent_but_later_realised',
    ])
)

confirmation_judge_payload.response.outcome_agrees_with_expected_outcome  confirmation_judge_payload.response.erroneous_info_given_by_eligibility_agent_without_realising  confirmation_judge_payload.response.erroneous_info_given_by_eligibility_agent_but_later_realised
True                                                                      False                                                                                            False                                                                                               29
False                                                                     True                                                                                             False                                                                                               20
                                                                          False                                                                                            False        

In [ ]:
pd.set_option('display.max_colwidth', None)

The agent correctly assessed the basic eligibility criteria for Child Benefit (responsibility, child's age, residency, and living arrangements) and correctly concluded that the applicant is eligible to apply. However, the agent failed critically by not investigating income levels or mentioning the High Income Child Benefit Charge (HICBC).\n\nThe expected outcome is "Eligible (but subject to tax charge)" which indicates this is the "High Earner" test case where one partner earns £75,000 per year - well over the £60,000 threshold for HICBC. The agent should have:\n1. Asked about the income of both partners\n2. Identified that income over £60,000 triggers a tax charge\n3. Explained that at £75,000, they would face a significant HICBC (approximately 75% of the benefit would be clawed back via tax)\n4. Mentioned the option to claim but opt out of receiving payments\n\nBy giving an unqualified "you would be eligible" response without addressing the tax charge implications, the agent provided misleading information by omission. The applicant would face an unexpected tax bill if they followed this advice without understanding the HICBC consequences. This is a material omission that makes the guidance incomplete and potentially harmful to the user's financial situation.

The agent's response partially agrees with the expected outcome but contains a critical omission that constitutes erroneous information by omission. \n\nThe agent correctly determined that the parent is currently eligible for Child Benefit (which aligns with "Eligible until week 12" since Tom has only been in residential accommodation for 10 weeks). However, the agent failed to mention the crucial 12-week time limit that applies to children in residential accommodation paid for by the local council.\n\nAccording to the expected outcome and the test case criteria, children in residential accommodation retain eligibility for only 12 weeks, after which eligibility ceases. Tom has been there for 10 weeks, meaning the parent has only 2 weeks of eligibility remaining.\n\nThe agent's statement "You should be able to apply for or continue receiving Child Benefit for Tom" is misleading because it suggests ongoing, indefinite eligibility without mentioning this imminent time limit. This is a material omission that would leave the parent uninformed about a critical aspect of their eligibility status.\n\nThe agent gave erroneous information by presenting eligibility as open-ended when it is actually time-limited and about to expire. This failure to communicate the 12-week rule means the agent did not provide complete or accurate guidance.

The agent reached a partially correct conclusion but failed in critical ways. While the basic eligibility determination is technically correct (a minor CAN claim Child Benefit for their own child), the agent failed to:\n\n1. **Ask about the applicant's age** - This is essential information that was never gathered, despite being crucial to the scenario.\n\n2. **Provide important guidance about financial implications** - The expected outcome specifically notes this is "Eligible (Though potentially financially suboptimal)." The test case clearly states that "You'll usually get more if [someone responsible for you] claims... on your behalf" and that if the minor claims, their parents might lose the Child Benefit they get for the minor themselves.\n\n3. **Give complete advice** - The agent should have informed the applicant about alternative claiming arrangements that would be more financially beneficial.\n\nThe agent's response is misleading by omission. While not technically incorrect in stating eligibility, it fails to provide the nuanced guidance that this specific situation requires. A proper assessment would acknowledge eligibility but strongly advise considering having a parent or guardian claim instead for better financial outcomes. The agent gave an incomplete picture that could lead to a financially suboptimal decision, which constitutes a failure to properly serve the user's interests.

In [ ]:
transformed_structured_df = structured

In [ ]:

analyse_cohort(transformed_structured_df, 'child_benefit', output_dir)